Jason Pettinato
jpettinato@umass.edu


# Building your own automatic differentiation

In this homework, you will implement automatic differentiation.

To set up your assignment, use the following code. This will define a set of primitive functions through which you can define expression graphs.

In [3]:
import operator

add = operator.add
sub = operator.sub
mul = operator.mul
div = operator.truediv
primitives = [add, sub, mul, div]


**Question 1**: Implement forward propagation. Define the following function.

```python
def fw_prop(x, funs, parents):
    """Inputs:
    x: A list of floating point values, the inputs to this function.
    funs: A list of functions, each an element of basic funs.
    parents: A list of pairs of parents, each an integer.
    """
    # do stuff
    return final_value # a single scalar
```

This function should do forward propagation, and return the final scalar output.

*Hint:* Your instructor’s implementation is less than 10 lines long.

To understand this function, suppose that it is called on the following inputs:

- `x=[2.0, 3.0]`
- `funs=[add]`
- `parents=[[0,1]]`

Then the expression graph simply defines the function `x[0]+x[1]` and so should return 5.

On the other hand, suppose the function is called on the following inputs:

- `x=[2.0, 5.0]`
- `funs=[add, mul]`
- `parents=[[0,1],[0,2]]`

Then the expression graph defines the function `x[0]*(x[0]+x[1])`  and should return 14.

On the other hand, suppose the function is called on the following inputs:

- `x=[1.0, 1.0]`
- `funs=[add, add, add, add, add]`
- `parents=[[0,1],[1,2],[2,3],[3,4],[4,5]]`

Then the expression graph defines successive terms in the Fibonnaci sequence and should return 13.

For your solution, you only need to provide your code. Provide your code directly in your solution PDF.

In [5]:

def fw_prop(x, funs, parents):
    """Inputs:
    x: A list of floating point values, the inputs to this function.
    funs: A list of functions, each an element of basic funs.
    parents: A list of pairs of parents, each an integer.
    """

    for idx in range(len(funs)):
        parent = parents[idx]
        fun = funs[idx]
        x1 = x[parent[0]]
        x2 = x[parent[1]]

        x_operation = fun(x1,x2)
        x.append(x_operation)
    
    final_value = x[-1]
    return final_value # a single scalar


**Question 2**: Run the following code:

```python
x = [1.1, 2.2, 3.3]
parents = [[0,2],[1,3],[2,4], [4,5]]
funs = [mul, div, pow, sub]
fw_prop(x, funs, parents)
```

What does your code output? All you need to provide is a number.

In [6]:
x = [1.1, 2.2, 3.3]
parents = [[0,2],[1,3],[2,4], [4,5]]
funs = [mul, div, pow, sub]
fw_prop(x, funs, parents)

-1.455759985551453


**Question 3**: Implement finite difference gradients. You will exploit the fact that

$$
\frac{\partial f(x)}{\partial x_i} \approx \frac{1}{2 \epsilon} (f(x+\epsilon \hat{e}_i)-f(x-\epsilon \hat{e}_i)),
$$

where $\hat{e}_i$ indicates the indicator function in the i-th dimension. Define the following functoin

```python
def finite_diff_grad(x, funs, parents):
    """Inputs:
    x: A list of floating point values, the inputs to this function.
    funs: A list of functions, each an element of basic funs.
    parents: A list of pairs of parents, each an integer.
    """
    epsilon = 1e-5
    # do stuff
    return grad # A list of floats, same length as x
```

When you implement this function, you should call `fw_prop`  2*D times, where D is the length of `x`. (This function is quite slow for large inputs, which is why we prefer to use backpropagation!)

For your solution, you only need to provide your code. Provide your code directly in your solution PDF.

In [9]:
def finite_diff_grad(x, funs, parents):
    """Calculate gradient using finite differences"""
    epsilon = 1e-5
    grad = []
    
    # For each input value in x
    for i in range(len(x)):
        # Create x + epsilon*e_i
        x_plus = x.copy()
        x_plus[i] += epsilon
        
        # Create x - epsilon*e_i
        x_minus = x.copy()
        x_minus[i] -= epsilon
        
        # Calculate (f(x + epsilon*e_i) - f(x - epsilon*e_i)) / (2*epsilon)
        f_plus = fw_prop(x_plus.copy(), funs, parents)
        f_minus = fw_prop(x_minus.copy(), funs, parents)
        
        partial_derivative = (f_plus - f_minus) / (2 * epsilon)
        grad.append(partial_derivative)
    
    return grad


**Question 4**: Run the following code

```python
x = [1.1, 2.2, 3.3]
parents = [[0,2],[1,3],[2,4], [4,5]]
funs = [mul, div, div, sub]
finite_diff_grad(x, funs, parents)
```

What does your code output? All you need to provide is a list of numbers.

In [10]:
x = [1.1, 2.2, 3.3]
parents = [[0,2],[1,3],[2,4], [4,5]]
funs = [mul, div, div, sub]
finite_diff_grad(x, funs, parents)

[-5.500964187410417, 2.750482093727413, -3.4836547291305915]


**Question 5**: Implement backpropagation. You should implement the following function: 

```python
def bw_prop(x, funs, parents):
    """Inputs:
    x: A list of floating point values, the inputs to this function.
    funs: A list of functions, each an element of basic funs.
    parents: A list of pairs of parents, each an integer.
    """
    # do stuff
    return grad # A list of floats, same length as x
```

Unlike finite differences, this code should have complexity only around twice as slow as `fw_prop`, no matter how big the function.

*Hint:* Your instructor’s implementation is less than 35 lines long.

For your solution, you only need to provide your code. Provide your code directly in your solution PDF.

In [23]:
def bw_prop(x, funs, parents):
    """Inputs:
    x: A list of floating point values, the inputs to this function.
    funs: A list of functions, each an element of basic funs.
    parents: A list of pairs of parents, each an integer.
    """
    for idx in range(len(funs)):
        parent = parents[idx]
        fun = funs[idx]
        x1 = x[parent[0]]
        x2 = x[parent[1]]

        x_operation = fun(x1,x2)
        x.append(x_operation)
    
    # Initialize gradients
    grads = [0] * len(x)  
    grads[-1] = 1    
    
    # Backward pass; this is reverse indexing
    for idx in range(len(funs) - 1, -1, -1):
        parent1, parent2 = parents[idx]
        fun = funs[idx]
        x1, x2 = x[parent1], x[parent2]
        current_idx = idx + len(x) - len(funs)
        grad_output = grads[current_idx]
        
        # Compute partial derivatives based on the operation
        if fun == mul:
            grads[parent1] += grad_output * x2
            grads[parent2] += grad_output * x1
        elif fun == div:
            grads[parent1] += grad_output / x2
            grads[parent2] += grad_output * (-x1 / (x2 * x2))
        elif fun == add:
            grads[parent1] += grad_output
            grads[parent2] += grad_output
        elif fun == sub:
            grads[parent1] += grad_output
            grads[parent2] += -grad_output
    
    return grads[:len(x) - len(funs)]


**Question 6**: Run the following code:

```python
x = [1.1, 2.2, 3.3]
parents = [[0,2],[1,3],[2,4], [4,5]]
funs = [mul, div, div, sub]
bw_prop(x, funs, parents)
```

What does your code output? All you need to provide is a list of numbers.

*Hint*: You should get something very close to what you got with finite differences. If you don’t, you have an error!

In [24]:

x = [1.1, 2.2, 3.3]
parents = [[0,2],[1,3],[2,4], [4,5]]
funs = [mul, div, div, sub]
bw_prop(x, funs, parents)

[-5.500964187327824, 2.7504820936639116, -3.4836547291092748]